# 1.1 Processing Mechanisms

*Installation and Setup*

In [1]:
%%capture
!pip install psyneulink

In [2]:
import psyneulink as pnl

*[Processing Mechanisms](https://princetonuniversity.github.io/PsyNeuLink/ProcessingMechanism.html#module-psyneulink.core.components.mechanisms.processing.processingmechanism)* are the basic units of computation in PsyNeuLink. At their core is a parameterized *function,* but they also contain the machinery to interact with input, output, control, and learning signals. Here, we will introduce the basic concept of two commonly used `ProcessingMechanisms`:

- [Transfer Mechanisms](https://princetonuniversity.github.io/PsyNeuLink/TransferMechanism.html), which perform a transformation on their input.
- [Integrator Mechanisms](https://princetonuniversity.github.io/PsyNeuLink/IntegratorMechanism.html), which maintain a hidden state that is integrated over time.

## Transfer Mechanisms

Here, we introduce [Transfer Mechanisms](https://princetonuniversity.github.io/PsyNeuLink/TransferMechanism.html). There are other types of mechanisms as well, such as [Integrator Mechanisms](https://princetonuniversity.github.io/PsyNeuLink/IntegratorMechanism.html), which maintain a hidden state that is integrated over time. We will cover those later.

To begin, we’ll define a simple `TransferMechanism` that performs a linear transformation on a scalar input. By default, this uses the identity function, meaning the input is passed through unchanged:

In [4]:
identity_mechanism = pnl.TransferMechanism()

We can execute mechanisms with the `execute` method. It takes an input and returns the output of the mechanism:

In [5]:
identity_mechanism.execute(5)

array([[5.]])

### Functions

We can change the function of the mechanism by setting its `function` attribute. For example, we can set it to a linear function with a different slope and intercept:

In [13]:
linear_mechanism = pnl.TransferMechanism(function=pnl.Linear(slope=2, intercept=1))
linear_mechanism.execute(5)

array([[11.]])

PsyNeuLink provides a variety of often-used functions, such as `Logistic`, `Exponential`, and `SoftMax` (see, [Transfer Functions](https://princetonuniversity.github.io/PsyNeuLink/TransferFunctions.html#overview)). For example, we can set the function to be a logistic function:

In [17]:
logistic_mechanism = pnl.TransferMechanism(function=pnl.Logistic(gain=2, offset=1))
logistic_mechanism.execute(5)

array([[1.9999546]])

#### Custom Functions

We can also use custom functions. For example, we can define a function and then wrap it in a `UserDefinedFunction` to use it as a mechanism's function:

In [43]:
def threshold_square(x):
    _x = x ** 4
    if _x < 5:
        return _x
    else:
        return 4

threshold_square_wrapped = pnl.UserDefinedFunction(threshold_square)
custom_mechanism = pnl.TransferMechanism(function=threshold_square_wrapped)
print(custom_mechanism.execute(1))
print(custom_mechanism.execute(2))
print(custom_mechanism.execute(3))

[[1.]]
[[4]]
[[4]]


### Input Shapes

In addition to the function, a `TransferMechanism` can be configured with an `input_shapes` argument, which specifies the shape of the input it expects. This is useful for mechanisms that need to handle multi-dimensional data. In this case the function will be applied to each element of the input independently.

In [44]:
linear_mechanism_2d = pnl.TransferMechanism(input_shapes=2)
linear_mechanism_2d.execute([1, 2])

array([[1., 2.]])

In addition to multidimensional inputs, a `TransferMechanism` can also handle multiple input ports. Each of the port will have its own shape. For example, in the following example, the mechanism has two input ports, one with shape `(2,)` and the other with shape `(1,)`. The function will be applied to each port independently:

In [85]:
linear_mechanism_2x3d = pnl.TransferMechanism(input_shapes=(2, 1), function=pnl.Linear(slope=2))
linear_mechanism_2x3d.execute([[1, 2], [3]])

array([array([2., 4.]), array([6.])], dtype=object)

## Integrator Mechanisms

An [Integrator Mechanisms](https://princetonuniversity.github.io/PsyNeuLink/IntegratorMechanism.html) integrates its input, possibly based on its prior values (hidden state). Instead of `TransferFunctions`, it uses [IntegratorFunctions](https://princetonuniversity.github.io/PsyNeuLink/IntegratorFunctions.html#psyneulink.core.components.functions.stateful.integratorfunctions.IntegratorFunction). Most of these functions use the hidden state of the mechanism and 'integrate' the input with the current state.

For example the `SimpleIntegrator` function adds the input to the hidden state:



In [6]:
integrator_mechanism = pnl.IntegratorMechanism(
    function=pnl.SimpleIntegrator()
)
# The initial value of the mechanism is 0, which is the default hidden state.
integrator_mechanism.value

array([[0.]])

In [14]:
# if we run it once with the input 1, the result (and the hidden state) is 1:
print(integrator_mechanism.execute(1)) # <-- this is 1

[[7.]]


In [13]:
# if we run it another time, the value and result "accumulates":
print(integrator_mechanism.execute(1)) # <-- this is 2 = 1 (hidden) + 1 (input)

[[6.]]


In [12]:
# and another time:
print(integrator_mechanism.execute(1)) # <-- this is 3 = 2 (hidden) + 1 (input)

res: [[5.]]
value: [[5.]]


In [20]:
integrator_mechanism_adaptive = pnl.IntegratorMechanism(
    function=pnl.AdaptiveIntegrator(
        rate=.3
    )
)
# The initial value of the mechanism is 0, which is the default hidden state.
integrator_mechanism_adaptive.value

array([[0.]])

In [21]:
print(integrator_mechanism_adaptive.execute(1)) # <-- 0.3 = (1-.7) * 0 (hidden state) + .3 * 1 (input)

[[0.3]]


In [22]:
print(integrator_mechanism_adaptive.execute(1)) # <-- 0.51 = (1-.7) * 0.3 (hidden state) + .3 * 1 (input)

[[0.51]]


In [23]:
print(integrator_mechanism_adaptive.execute(1)) # <-- 0.657 = (1-.7) * 0.3 (hidden state) + .3 * 1 (input)


[[0.657]]


In this tutorial we only discussed [Processing Mechanisms](https://princetonuniversity.github.io/PsyNeuLink/ProcessingMechanism.html), which are a subclass of the broader class of [Mechanisms](https://princetonuniversity.github.io/PsyNeuLink/Mechanism.html). PsyNeuLink also includes another type of mechanisms, specifically [Modulatory Mechanisms](https://princetonuniversity.github.io/PsyNeuLink/ModulatoryMechanism.html) such as [Control Mechanisms](https://princetonuniversity.github.io/PsyNeuLink/ControlMechanism.html) and [Learning Mechanisms](https://princetonuniversity.github.io/PsyNeuLink/LearningMechanism.html). While `ProcessingMechanisms` are used to *process* inputs and pass them to outputs, `ModulatoryMechanisms` are used to *modulate* this processing. For example, instead of being fixed, the `rate` parameter of the `AdaptiveIntegrator` could be modulated by a `ControlMechanism` to change the integration rate based on the current state of the system. We will cover these `ModulatoryMechanisms` in later tutorials.